In [1]:
import numpy  as np
import pandas as pd
import tensorflow as tf

In [2]:
tf.__version__

'2.10.1'

# Import the dataset

In [3]:
data = pd.read_csv('Churn_Modelling.csv')

In [4]:
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


# Data preprocessing

### check there is any missing values

In [5]:
data.isnull().sum()

RowNumber          0
CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

In [6]:
X = data.iloc[:, 3:-1].values
y = data.iloc[:, -1].values

In [42]:
data['Geography'].unique()

array(['France', 'Spain', 'Germany'], dtype=object)

In [37]:
np.set_printoptions(threshold=np.inf) ## Set print options to display the full row


In [ ]:
print(X)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]


# Encording categorical Data

In [9]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])

In [10]:
print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


# One Hot Encording

In [11]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

In [12]:
print(X)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


## Make balance dataset

In [13]:
pd.Series(y).value_counts()

0    7963
1    2037
Name: count, dtype: int64

In [14]:
from imblearn.over_sampling import SMOTE
smote = SMOTE(sampling_strategy='auto')

In [15]:
X,y = smote.fit_resample(X,y)

In [16]:
pd.Series(y).value_counts()

1    7963
0    7963
Name: count, dtype: int64

# Train Test Split

In [17]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# Feature Scalling

In [18]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

 the StandardScaler is used to bring all features to a common scale

# Build the ANN

In [19]:
ann = tf.keras.models.Sequential()

In [20]:
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import BatchNormalization

Add the input layer and the first hidden layer

In [21]:
ann.add(Dense(256 , activation='relu',kernel_regularizer=l2(0.01)))
ann.add(BatchNormalization())
ann.add(Dropout(0.2))   

Add the second sencond hidden layer

In [22]:
ann.add(Dense(256 , activation='relu',kernel_regularizer=l2(0.01)))
ann.add(BatchNormalization())
ann.add(Dropout(0.2))

Add output layer

In [23]:
ann.add(tf.keras.layers.Dense(units=1 , activation='sigmoid'))

# Train the ANN

In [24]:
from tensorflow.keras.optimizers import Adam
learning_rate = 0.0001

In [25]:
ann.compile(optimizer = Adam(learning_rate=learning_rate), loss = 'binary_crossentropy', metrics = ['accuracy'])

In [26]:
from tensorflow.keras.callbacks import EarlyStopping
earlystopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1, mode='min')

In [27]:
ann.fit(X_train, y_train,
        validation_data=(X_test, y_test),
        batch_size = 32, epochs = 150,
        callbacks=[earlystopping])

Epoch 1/150
399/399 [==============================] - 4s 4ms/step - loss: 3.1247 - accuracy: 0.6844 - val_loss: 2.6789 - val_accuracy: 0.7994
Epoch 2/150
399/399 [==============================] - 2s 4ms/step - loss: 2.5115 - accuracy: 0.7630 - val_loss: 2.2140 - val_accuracy: 0.8189
Epoch 3/150
399/399 [==============================] - 2s 4ms/step - loss: 2.1321 - accuracy: 0.7819 - val_loss: 1.9065 - val_accuracy: 0.8233
Epoch 4/150
399/399 [==============================] - 1s 4ms/step - loss: 1.8572 - accuracy: 0.7957 - val_loss: 1.6709 - val_accuracy: 0.8271
Epoch 5/150
399/399 [==============================] - 1s 4ms/step - loss: 1.6381 - accuracy: 0.8024 - val_loss: 1.4902 - val_accuracy: 0.8308
Epoch 6/150
399/399 [==============================] - 1s 4ms/step - loss: 1.4743 - accuracy: 0.8061 - val_loss: 1.3394 - val_accuracy: 0.8374
Epoch 7/150
399/399 [==============================] - 2s 4ms/step - loss: 1.3330 - accuracy: 0.8153 - val_loss: 1.2130 - val_accuracy: 0.8399

Let's say you have 10,000 samples and a batch size of 32:

*   For the first 32 samples, the model processes them and adjusts its weights.

*   Then it processes the next 32 samples, adjusts the weights again, and so on, until it has processed the entire 10,000 samples.




100 epochs means the model will go through the entire dataset 100 times in total.




# Test the model

Use our ANN model to predict if the customer with the following informations will leave the bank:

Geography: France

Credit Score: 600

Gender: Male

Age: 40 years old

Tenure: 3 years

Balance: \$ 60000

Number of Products: 2

Does this customer have a credit card? Yes

Is this customer an Active Member: Yes

Estimated Salary: \$ 50000

So, should we say goodbye to that customer?

In [28]:
predict = (ann.predict(sc.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])) > 0.5).astype("int32")
if predict == 1:
    print("Customer will leave the bank")
else:
    print("Customer will not leave the bank")

1/1 [==============================] - 0s 96ms/step
Customer will not leave the bank


Therefore, our ANN model predicts that this customer stays in the bank!


# Prediction

In [29]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

100/100 [==============================] - 0s 855us/step
[[1 1]
 [1 1]
 [1 1]
 ...
 [0 0]
 [1 1]
 [0 0]]


In [30]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[1448  142]
 [ 228 1368]]


0.8838669177652229

In [44]:
ann.save('model.h5')
    

In [45]:
import pickle
with open('scaler.pkl','wb') as file:
    pickle.dump(sc,file)